In [1]:
import psycopg2
import pandas as pd
import pandas.io.sql as sqlio
import yaml
from pathlib import Path

In [ ]:
# from https://stackoverflow.com/a/63170705
def get_settings():
    full_file_path = Path(__file__).parent.joinpath("../config.yml")
    with open(full_file_path) as settings:
        settings_data = yaml.load(settings, Loader=yaml.Loader)
    return settings_data

In [2]:
config = get_settings()

    # connection establishment
conn = psycopg2.connect(
    database=config["db"]["database"],
    user=config["db"]["user"],
    password=config["db"]["password"],
    host=config["db"]["host"],
    port=config["db"]["port"],
)
conn.autocommit = True
cursor = conn.cursor()

In [3]:
# retrieve data in one big ass table
sql = '''
  select *
  from player p 
  inner join team t
	on p.team_abbreviation = t.team_id 
	and p.season = t.season;
'''
df = sqlio.read_sql_query(sql, conn)
df = df.T.drop_duplicates().T

/home/amtalb/.cache/pypoetry/virtualenvs/heisman-poI2x7aT-py3.10/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [16]:
# separate out training/test for time series data
train = df[df['season'] < 2005]
test = df[df['season'] == 2005]

train_X = train[['votes']]
train_y = train.loc[ : , train.columns != 'votes']
test_X = test[['votes']]
test_y = test.loc[ : , test.columns != 'votes']

In [17]:
from sklearn.linear_model import LinearRegression

regr = LinearRegression()
regr.fit(train_X, train_y)
print(regr.score(train_X, train_y))

ValueError: Input contains NaN

In [ ]:
# run a huge model grid to find which is best
# running regressions to predict heisman votes

In [ ]:
# close db
conn.commit()
conn.close()